<a href="https://colab.research.google.com/github/Deepti2412/Codes/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np

In [2]:
with open('/content/train_qa.txt','rb') as f:
      train_data=pickle.load(f)

In [3]:
with open('/content/test_qa.txt','rb') as f:
      test_data=pickle.load(f)

In [4]:
type(train_data)

list

In [5]:
len(train_data)

10000

In [6]:
len(test_data)

1000

In [7]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

#Creating a vocabulary

In [8]:
all_data=test_data + train_data

In [9]:
len(all_data)

11000

In [10]:
vocab=set()
for story,question,answer in all_data:
  vocab=vocab.union(set(story))
  vocab=vocab.union(set(question))
  


In [11]:
vocab.add('no')

vocab.add('yes')

In [12]:
vocab_len=len(vocab) + 1

In [13]:
vocab_len

38

In [14]:
all_story_lens=[len(data[0]) for data in all_data]

In [15]:
max_story_len=max(all_story_lens)

In [27]:
max_question_len=max([len(data[1]) for data in all_data])

In [28]:
max_question_len

6

In [18]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [19]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [ ]:
tokenizer.word_index

In [21]:
train_story_text=[]
train_question_text=[]
train_answer_text=[]

for story,question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answer_text.append(answer)

In [22]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)

In [35]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
  #Stories X
  X=[]
  #Question
  Xq=[]
  #answers
  Y=[]
  for story,query,answer in data:
    x=[word_index[word.lower()] for word in story]
    xq=[word_index[word.lower()] for word in query]
    y=np.zeros(len(word_index)+1)
    y[word_index[answer]]=1
    X.append(x)
    Xq.append(xq)
    Y.append(y)
  return (pad_sequences(X,maxlen=max_story_len),(pad_sequences(Xq,maxlen=max_question_length)),(np.array(Y)))

In [38]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [39]:
inputs_train,queries_train, answers_train

(array([[ 0,  0,  0, ..., 31, 26, 37],
        [ 0,  0,  0, ..., 31, 25, 37],
        [ 0,  0,  0, ..., 31,  8, 37],
        ...,
        [ 0,  0,  0, ..., 31, 26, 37],
        [ 0,  0,  0, ..., 13, 28, 37],
        [ 0,  0,  0, ...,  5, 28, 37]], dtype=int32),
 array([[32, 34, 36, 31, 25, 16],
        [32, 12, 36, 31,  8, 16],
        [32, 12, 36, 31, 27, 16],
        ...,
        [32, 34, 36, 31, 25, 16],
        [32, 22, 36, 31,  2, 16],
        [32, 22, 36, 31, 26, 16]], dtype=int32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]))

In [40]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [41]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout
from keras.layers import add, dot, concatenate
from keras.layers import LSTM

In [42]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [44]:
vocab_size=vocab_len

In [45]:
# Input gets embedded to a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

# This encoder will output:
# (samples, story_maxlen, embedding_dim)

In [46]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

In [47]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=max_question_len))
question_encoder.add(Dropout(0.3))

In [48]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [49]:
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

In [50]:
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [51]:
answer = concatenate([response, question_encoded])

In [54]:
answer

<tf.Tensor 'lstm_1/strided_slice_18:0' shape=(None, 32) dtype=float32>

In [53]:
answer = LSTM(32)(answer)  # (samples, 32)

In [55]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)

In [56]:
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [57]:
# train
history = model.fit([inputs_train, queries_train], answers_train,batch_size=32,epochs=120,validation_data=([inputs_test, queries_test], answers_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 5s 546us/step - loss: 0.9015 - accuracy: 0.5088 - val_loss: 0.6947 - val_accuracy: 0.5030
Epoch 2/120
10000/10000 [==============================] - 5s 465us/step - loss: 0.7049 - accuracy: 0.4949 - val_loss: 0.6949 - val_accuracy: 0.4970
Epoch 3/120
10000/10000 [==============================] - 5s 471us/step - loss: 0.6955 - accuracy: 0.5078 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 4/120
10000/10000 [==============================] - 5s 459us/step - loss: 0.6958 - accuracy: 0.4926 - val_loss: 0.6937 - val_accuracy: 0.4970
Epoch 5/120
10000/10000 [==============================] - 5s 459us/step - loss: 0.6950 - accuracy: 0.4951 - val_loss: 0.6932 - val_accuracy: 0.5030
Epoch 6/120
10000/10000 [==============================] - 5s 454us/step - loss: 0.6948 - accuracy: 0.4994 - val_loss: 0.6941 - val_accuracy: 0.5030
Epoch 7/120
10000/10000 [==============================] 

In [58]:
filename = 'chatbot_120_epochs.h5'
model.save(filename)

In [64]:
model.load_weights(filename)
pred_results = model.predict(([inputs_test, queries_test]))

In [65]:
test_data[0][0]

['Mary',
 'got',
 'the',
 'milk',
 'there',
 '.',
 'John',
 'moved',
 'to',
 'the',
 'bedroom',
 '.']

In [66]:
story =' '.join(word for word in test_data[0][0])
print(story)

Mary got the milk there . John moved to the bedroom .


In [67]:
query = ' '.join(word for word in test_data[0][1])
print(query)

Is John in the kitchen ?


In [68]:
print("True Test Answer from Data is:",test_data[0][2])

True Test Answer from Data is: no


In [69]:
val_max = np.argmax(pred_results[0])

for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

print("Predicted answer is: ", k)
print("Probability of certainty was: ", pred_results[0][val_max])

Predicted answer is:  no
Probability of certainty was:  0.9974075
